# finding reading all das files

In [9]:
%load_ext autotime
base_directory='/home/osman/aramko/'
das_directory=base_directory+'inputs/das'
rep=base_directory+'/outputs/das/h5_files/'

time: 1.01 ms


In [10]:
def h5_reader(f):
    return h5py.File(f, 'r')

def time_extracter(h5file):
    return int(h5file['t'][0])

def das_extracter(f):
    return pd.DataFrame(np.array(h5_reader(f).get('das')))

time: 1.7 ms


In [11]:
import glob
import pandas as pd
import h5py
import swifter
import numpy as np
import sys
from scipy import signal
files=glob.glob(das_directory+"/*.h5")
filesdf=pd.DataFrame(files,columns=['file'])
chanels=list(h5_reader(filesdf['file'][0])['channel'])
filesdf['name']=filesdf['file'].swifter.apply(lambda x:x.split('/')[-1].replace('.h5',''))
chanels=list(h5_reader(filesdf['file'][0])['channel'])
len(files)

918

time: 114 ms


# finding missed data in das

In [12]:
def filename_extracter(last_file_name):
        if last_file_name[-2:]=='17':
            return str(int(last_file_name)+30)
        else :
            return str(int(last_file_name)+70)
#we noticed the wasy the data file name is ....         

time: 1.63 ms


In [13]:
filesdf['start']=filesdf['file'].swifter.apply(lambda x:time_extracter(h5_reader(x)))
filesdf=filesdf.sort_values('start').reset_index(drop=True)
all_files=pd.DataFrame(list(range(filesdf.start.min(),filesdf.start.max(),30)),columns=['start']).copy()
all_files=all_files.merge(filesdf,how='left',on='start')
all_files['last_file_name']=all_files['name'].shift(periods=1)
all_files=all_files.fillna('-1')
all_files['output']=0
for i in all_files[all_files.name=='-1'].index:
    all_files.loc[i, 'name'] = filename_extracter(all_files.loc[i, 'last_file_name'])
    all_files.loc[i+1, 'last_file_name']=all_files.loc[i, 'name']
    all_files.loc[i, 'output']=1
all_files[all_files.output==1][['start','name']] #thats the time missing parts with the names generated

,start,name
483,1458306107,160318130147
484,1458306137,160318130217
485,1458306167,160318130247
486,1458306197,160318130317
487,1458306227,160318130347
...,...,...
1081,1458324047,160318176047
1082,1458324077,160318176117
1083,1458324107,160318176147
1084,1458324137,160318176217


time: 2.75 s


# splitting das files by chanels

In [6]:
numsimp=int(30000/100)
def signal_preprocessing(sig):
    lowpass_filter = signal.butter(1,Wn=0.001,btype='low',output='sos')
    a, b = signal.welch(signal.resample((signal.sosfilt(lowpass_filter,sig)),int(numsimp)), fs=0.1, scaling='spectrum')
    return b

def preprocessing_generale(file):    
    data=[]
    tmp=das_extracter(file)
    for i in tmp.columns:
        data.append(signal_preprocessing(tmp[i]))
    result=pd.DataFrame([data])
    result['file']=file
    return result


time: 576 µs


In [7]:
import multiprocessing
import numpy as np
import pandas as pd
num_cores = multiprocessing.cpu_count()
with multiprocessing.Pool(num_cores) as pool:
    data_by_chanel = pd.concat(pool.map(preprocessing_generale, filesdf['file']), ignore_index=True)

time: 3min 36s


# starting processing data by channel for TRAINING

In [8]:
from sklearn.preprocessing import MinMaxScaler
from cuml.decomposition import PCA as cuPCA
from cuml import DBSCAN as cumlDBSCAN
scaler= MinMaxScaler()
pca_cuml = cuPCA(n_components=3,random_state=42)
def signal_transform(filesdf,chanel):
    result_cuml = pca_cuml.fit_transform(scaler.fit_transform(pd.DataFrame(filesdf[chanel].values.tolist())))
    data_by_chanel[chanel]=pd.DataFrame(pd.DataFrame(result_cuml).apply(lambda r: tuple(r), axis=1).apply(np.array))
    return result_cuml

time: 4.26 s


In [9]:
# appliying PCA on the signale 
dataset=np.concatenate([signal_transform(data_by_chanel,i) for i in range(len(chanels))],axis=0)

time: 20.4 s


In [10]:
#small trick to access rapidly to the missed data
all_files=all_files.merge(data_by_chanel,how='left',on='file')

time: 45.1 ms


In [11]:
from sklearn.model_selection import train_test_split
def shifter(dataset,look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
            a = dataset[i:(i + look_back),:]
            dataX.append(a)
            dataY.append(dataset[i + look_back, :])
    x=np.asarray(dataX).reshape(len(dataX),look_back,len(dataY[1])) 
    y=np.asarray(dataY).reshape(len(dataY),len(dataY[1]))
    return train_test_split(x, y, test_size=0.2, random_state=42)
def preprocessing_training(chanels):
    window=20
    x_train, x_test, y_train, y_test = shifter(dataset,window)
    model= trainer(window,x_train, x_test, y_train, y_test)
    return model

time: 1.9 ms


In [14]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.optimizers import  Adam,RMSprop
from keras.callbacks import TensorBoard
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)
rm=RMSprop(
    learning_rate=0.005,
)
def trainer(window,x_train, x_test, y_train, y_test):
    shape=3
    size=300
    drops=0.2
    model = Sequential()
    model.add(LSTM(size,input_shape=(window,shape),return_sequences=True))
    model.add(Dropout(drops))
    model.add(LSTM(size,return_sequences=False))
    model.add(Dropout(drops))
    model.add(Dense(shape))
    model.compile(loss='mae', optimizer=rm,metrics=['cosine_similarity'])
    model.fit(x_train, y_train, epochs=50,verbose=2,validation_split=0.2,batch_size=600)
    print('model score : ',model.evaluate(x_test, y_test,verbose=2)[1])
    return model

time: 11.2 ms


# TRAINING

In [15]:
model=preprocessing_training(range(len(chanels)))

Epoch 1/50
376/376 - 19s - loss: 0.1049 - cosine_similarity: 0.5185 - val_loss: 0.0888 - val_cosine_similarity: 0.5942
Epoch 2/50
376/376 - 18s - loss: 0.0890 - cosine_similarity: 0.6019 - val_loss: 0.0836 - val_cosine_similarity: 0.6234
Epoch 3/50
376/376 - 18s - loss: 0.0851 - cosine_similarity: 0.6295 - val_loss: 0.0801 - val_cosine_similarity: 0.6447
Epoch 4/50
376/376 - 18s - loss: 0.0823 - cosine_similarity: 0.6493 - val_loss: 0.0785 - val_cosine_similarity: 0.6663
Epoch 5/50
376/376 - 18s - loss: 0.0797 - cosine_similarity: 0.6682 - val_loss: 0.0763 - val_cosine_similarity: 0.6737
Epoch 6/50
376/376 - 18s - loss: 0.0772 - cosine_similarity: 0.6856 - val_loss: 0.0744 - val_cosine_similarity: 0.6904
Epoch 7/50
376/376 - 18s - loss: 0.0750 - cosine_similarity: 0.7014 - val_loss: 0.0714 - val_cosine_similarity: 0.7060
Epoch 8/50
376/376 - 18s - loss: 0.0727 - cosine_similarity: 0.7172 - val_loss: 0.0705 - val_cosine_similarity: 0.7176
Epoch 9/50
376/376 - 18s - loss: 0.0708 - cosine

# starting prediction of lost data 

In [16]:
def extract_rows(end,window=20,features=3):
    all_results=[]
    segment=all_files[end-window:end][range(len(chanels))].copy().reset_index(drop=True)
    for ch in range(len(chanels)):
        try:
            all_results.append(np.concatenate([segment[ch][i] for i in range(len(segment[ch]))],axis=0).reshape(1,window,features))
        except:
            print (segment)   
            
    return np.concatenate([i for i in all_results],axis=0)

def restorer(chanel,i,resutls):
    all_files[chanel].loc[i]=resutls[chanel].copy()

def predicter(end,window=20,features=3):
    resutls=model.predict(extract_rows(end))
    for ch in range(len(chanels)):  
        restorer(ch,end,resutls)  


time: 2.09 ms


In [17]:
for i in all_files[all_files['output']==1].index:
    predicter(i) #predict all the missed data one by one

/home/osman/anaconda3/envs/rapids-0.16/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


time: 44.1 s


### NearestNeighbors algorithm used to make the new generted data like the same in dataset to avoid outliers 

In [18]:
from cuml.neighbors import NearestNeighbors
def get_nearest(chanel) :
    NN=NearestNeighbors(n_neighbors=1)
    NN.fit(pd.DataFrame(extract[chanel].values.tolist()))
    output[chanel]=NN.kneighbors(pd.DataFrame(output[chanel].values.tolist()),return_distance=False)

time: 480 µs


### generating template for missing data in das with the names they should be

In [21]:
all_files['file']=all_files.swifter.apply(lambda x:generating_missed_files(filesdf['file'][0], x['name'],x['start']) if x['output']==1 else x['file'],axis=1)


time: 24.3 s


In [29]:
output=all_files[all_files['output']==1].copy().reset_index(drop=True)
for ch in  range(len(chanels)):get_nearest(ch)

time: 44.8 s


In [20]:
def generating_missed_files(exemple, dest,t,rep=rep):
    dest=rep+dest+'.h'
    with h5py.File(exemple, 'r') as f1:
        with h5py.File(dest, 'w') as f2:
            for ds in f1.keys():
                f1.copy(ds, f2)
            f2['t'][:]=np.array(range(t,t+30000))
    f2.close()
    f1.close()
    return dest


time: 1.85 ms


# writing final results data 

In [24]:
def das_ins(index,chanel):
    with h5py.File(output.loc[index]['file'], 'r+') as f1:
        with h5py.File(filesdf['file'][output.loc[index][chanel]], 'r') as f2:
            f1['das'][:,chanel]=f2['das'][:,chanel]
    f1.close()
    f2.close()

time: 1.94 ms


In [ ]:
import multiprocessing
from functools import partial
for i in range(len(chanels)):
    with multiprocessing.Pool(4) as pool:
        pool.map(partial(das_ins, chanel=i), range(len(output)))
    print(i)